# Homework 3: Language Modelling in Hangman

Student Name: Saransh Srivastava

Student ID: 1031073

## General info

<b>Due date</b>:  Friday, 17 May 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models. Your objective is to create an automatic player which makes the fewest mistakes.

## The Hangman Game (*No implementation is needed*)

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. 

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided gusser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('Enter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to True. When submitting your solution, set to False so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [4]:
if interactive:
    hangman('whatever', human, 8, True)

## 1. Preparing Test Set and Training Set (1 mark)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the algorithm. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown` and the `words` method, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(1 mark)

In [5]:
from nltk.corpus import brown
import numpy as np

np.random.seed(12345)

# word_set stores all the unique word types in the Brown corpus
word_set = []
# test_set stores 1000 word types for testing
test_set = []
# training_set stores the rest word types for training
training_set = []

###
# Your answer BEGINS HERE
###
count =0
for word in brown.words():
    word = word.lower()
    if word.isalpha():
        word_set.append(word)
word_set = list(set(word_set))
         
np.random.shuffle(word_set)
for word in word_set:        
    if count < 1000:
        test_set.append(word)
    else:
        training_set.append(word)
    count +=1

###
# Your answer ENDS HERE
###

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


<b>For your testing:</b>

In [6]:
assert(len(word_set) > 35000 and len(word_set) < 45000)

In [7]:
assert(len(test_set) == 1000)

In [8]:
assert(len(training_set) + len(test_set) == len(word_set))

In [9]:
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

## 2. Simple Guesser: Random Guessing (1 mark)

<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

(1 mark)

In [10]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [11]:
import string

def random_guesser(mask, guessed, **kwargs):
    """
        This function implements a random guesser. It returns the random guess. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    while(True):
        letter = np.random.choice(list(string.ascii_lowercase), size=1, replace=False)[0]
        if letter not in guessed:
            return letter
    ###
    # Your answer ENDS HERE
    ###

# uncomment to run a single hangman game with output shown (useful for debugging)
#hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  16.804


<b>For your testing:</b>

In [12]:
assert(result > 10 and result < 20)

## 3. Your First AI Guesser: Unigram Guesser (1 mark)

**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

Hint: It should be much lower than random guessing.

(1 mark)

In [13]:
from collections import Counter

# unigram_counts stores the frequencies of characters over all training words
unigram_counts = Counter()

###
# Your answer BEGINS HERE
###
for word in training_set:
    unigram_counts += Counter(list(word))

###
# Your answer ENDS HERE
###

print(unigram_counts)

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        This function implements a unigram guesser. It returns a guess based on the unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    count = 0
    while(True):
        letter = unigram_counts.most_common(count+1)[count][0]
        count +=1
        if letter not in guessed:
            return letter

    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("Average number of incorrect guesses: ", result)

Counter({'e': 35158, 'i': 26059, 'a': 24363, 's': 23710, 'n': 22603, 'r': 22130, 't': 20639, 'o': 19158, 'l': 16650, 'c': 12597, 'd': 12275, 'u': 10191, 'm': 8684, 'g': 8611, 'p': 8552, 'h': 7351, 'b': 5737, 'y': 5346, 'f': 4204, 'v': 3438, 'k': 2987, 'w': 2804, 'z': 1025, 'x': 826, 'j': 644, 'q': 529})

Average number of incorrect guesses:  10.326


# <b>For your testing:</b>

In [14]:
assert(result > 5 and result < 15)

## 4. Your Second AI Guesser: Length-based Unigram Guesser (1 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having a *different* unigram model for each length of the words. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. You need to decide on how to handle this situation.

(1 mark)

In [15]:
from collections import defaultdict

# unigram_counts_by_length stores a dictionary, mapping word length to the frequencies of characters of words with that word length
unigram_counts_by_length = defaultdict(Counter)

###
# Your answer BEGINS HERE
###
for word in training_set:
    unigram_counts_by_length[len(word)] += (Counter(list(word)))

for key, value in unigram_counts_by_length.items():
    for letter in list(string.ascii_lowercase):
        if letter not in value.keys():
            unigram_counts_by_length[key][letter] = 0
    

###
# Your answer ENDS HERE
###

lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1
print(lengths)
print()
print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    """
        This function implements a length-based unigram guesser. It returns a guess based on the length-based unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    count = 0
    mask_len = len(mask)
    while(True):
        if mask_len > max_length:
            mask_len = max_length
        letter = unigram_counts_by_length[mask_len].most_common(count+1)[count][0]
        count +=1
        if letter not in guessed:
            return letter
    
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("Average number of incorrect guesses: ", result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

defaultdict(<class 'collections.Counter'>, {4: Counter({'e': 900, 'a': 866, 's': 698, 'o': 679, 'l': 557, 'i': 541, 'r': 514, 't': 489, 'n': 471, 'd': 363, 'u': 348, 'm': 296, 'p': 287, 'h': 286, 'c': 257, 'b': 256, 'k': 221, 'g': 221, 'w': 194, 'y': 176, 'f': 168, 'v': 97, 'j': 63, 'z': 45, 'x': 35, 'q': 8}), 7: Counter({'e': 5303, 'a': 3553, 's': 3459, 'r': 3423, 'i': 3341, 'n': 3040, 't': 2655, 'o': 2508, 'l': 2490, 'd': 2054, 'c': 1680, 'u': 1503, 'g': 1495, 'm': 1232, 'p': 1228, 'h': 1109, 'b': 944, 'y': 747, 'f': 667, 'k': 586, 'w': 520, 'v': 469, 'z': 153, 'x': 112, 'j': 97, 'q': 82}), 2: Counter({'a': 26, 'e': 20, 'm': 19, 'o': 18, 's': 18, 'i': 16, 'l': 13, 'h': 12, 'p': 12, 'u': 11, 't': 11, 'c': 11, 'd': 11, 'n': 10, 'f': 10, 'r': 9, 'b': 8, 'w': 7, 'v': 7, 'y': 6, 'j': 5, 'g': 5, 'k': 5, 'x': 3, 'q': 2, 'z': 1}), 10: Counter({'e': 4323, 'i': 3607, 'n': 3069, 's': 2891, 't': 2792, 'a': 2769, 'r'

<b>For your testing:</b>

In [16]:
assert(result > 5 and result < 15)

## 5. Your Third AI Guesser: Bigram Guesser (1 mark)

**Instructions:** Now for the next challenge, using a *bigram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

You should develop a *bigram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with a sentinel symbol `$`.) You should use *linear interpolation* to smooth between the higher order and lower order models, and you will have to decide how to weight each component (be reminded that all probabilities should sum to 1).

Your bigram guesser should apply your language model to each blank position in the secret word by using the left context as is known. E.g., in the partial word `$ _ e c _ e _ _` we know the left context for the first three blanks, but have no known left context for the last blank. Using a bigram language model, we are able to apply it to the first three blanks only. You should then select the character with the highest probability of predicting the most number of correct entries over the blanks. 

Do you see any improvement over the unigram guessers above?

(1 mark)

In [17]:
###
# Your answer BEGINS HERE
###
from numpy.random import choice 
import random
import operator

bigram_counts = None # you will want a different data structure to store the bigram 

def convert_sentence(word):
    result = ["<s>"] + [w.lower() for w in word]
    return result


def get_counts(sentences):
    bigram_counts = defaultdict(Counter)
    for sentence in sentences:
        sentence = convert_sentence(sentence)
        bigram_list = zip(sentence[:-1], sentence[1:])
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
    return bigram_counts

bigram_counts = get_counts(training_set)

for key, value in bigram_counts.items():
    for letter in list(string.ascii_lowercase):
        if letter not in value.keys():
            bigram_counts[key][letter] = 0

###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, counts=bigram_counts): # add extra default arguments if needed
    """
        This function implements a bigram guesser. It returns a guess based on the bigram model using linear interpolation.
    """
    ###
    # Your answer BEGINS HERE
    ###
    current_list = ["<s>"]
    for idx,blank in enumerate(mask):
        if blank != '_' and idx != len(mask):
            current_list.append(blank)
    mask = convert_sentence(mask)
    result = {}
    bigram_candidate = Counter()
    
    for item in set(current_list):
        bigram_candidate = bigram_candidate + bigram_counts[item]

    count3 = 0
    while(True):
        letter = bigram_candidate.most_common(count3+1)[count3][0]
        count3 +=1
        if letter not in guessed:
            return letter 
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), bigram_guesser, 10, True)

result = test_guesser(bigram_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  10.064


<b>For your testing:</b>

In [18]:
assert(result < 13)

## 6. Your Own AI Guesser (1 mark)

**Instructions:** You should try to develop a more effective AI, `my_ai_guesser`, for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, and using ngram models. Ensure you report the test performance of your guesser. Have fun!

You will be marked based on the explanation of your approach and its accuracy. 

(1 mark) 

In [19]:
###
# Your answer BEGINS HERE
###

def my_ai_guesser(mask, guessed):
    current_list = ["<s>"]
    for idx,blank in enumerate(mask):
        if blank != '_' and idx != len(mask):
            current_list.append(blank)
    mask = convert_sentence(mask)
    
    result = {}
    for item in set(current_list):
        indices = [i for i, x in enumerate(current_list) if x == item]
        for index in indices:
            if index < len(mask)-1 and mask[index+1] == '_':
                prev_word_counts = bigram_counts[item]
                total_counts = float(sum(prev_word_counts.values()))
                count = 0
                letter = prev_word_counts.most_common(count+1)[count][0]
                letter_val = prev_word_counts.most_common(count+1)[count][1]
                while letter in guessed:
                    count += 1
                    letter = prev_word_counts.most_common(count+1)[count][0]
                    letter_val = prev_word_counts.most_common(count+1)[count][1]
                result[letter] = letter_val/total_counts
    return max(result.items(), key=operator.itemgetter(1))[0]        
    

###
# Your answer ENDS HERE
###

result = test_guesser(my_ai_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  9.199


**Instructions:** Explain your approach and discuss your result below. Please keep it brief.

In my guesser, instead of taking the sum of distributions of left context key, I have compared the probability of occurance of a character after the left context. Based on whichever probability is higher, that becomes the next guess.
The approach reduces the incorrect guess more than the bigram vanilla model